In [1]:
import pandas as pd
import json
import requests
import os
from dotenv import load_dotenv

load_dotenv()
API_TOKEN = str(os.getenv('API_TOKEN'))

In [2]:
def read_json_file(path: str=None, url: str=None) -> dict:
    # with open(path, 'r') as f:
    #     return json.load(f)
    response = requests.get(url)
    data = response.json()
    return data

In [3]:
path = 'https://raw.githubusercontent.com/Ciencia-de-Dados-IMT-2023/Datasets/main/spotify_million_playlist_dataset/mpd.slice.0-999.json'
data = read_json_file(url=path)

In [4]:
df_musicas = pd.DataFrame()

# Itera sobre as playlists
for playlist in data['playlists']:
    name_playlist = playlist['name']
    id_playlist = playlist['pid']

    print(f'{id_playlist} - Playlist: {name_playlist}')

    # Itera sobre as músicas da playlist
    musicas_playlist = playlist['tracks']

    for musica in musicas_playlist:
        name_musica = musica['track_name']
        id_musica = musica['track_uri']

        # print(f'\tMúsica: {name_musica} - ID: {id_musica}')
        df_musica_aux = pd.DataFrame.from_dict(musica, orient='index')
        df_musica_aux = df_musica_aux.T

        # Adiciona o nome da playlist e o id da playlist
        df_musica_aux['playlist_name'] = name_playlist
        df_musica_aux['playlist_id'] = id_playlist

        df_musica_aux = df_musica_aux.T
        df_musicas = pd.concat([df_musicas, df_musica_aux], axis=1)
        
        # break
    break

df_musicas = df_musicas.T
# df_musicas.to_csv('data/df_musicas_parte_1.csv', index=False)
df_musicas.head(5)

0 - Playlist: Throwbacks


,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,playlist_name,playlist_id
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks,0
0,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,Throwbacks,0
0,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0
0,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,Throwbacks,0
0,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,Throwbacks,0


In [5]:
df_musicas = df_musicas[['track_uri', 'artist_uri', 'album_uri']]

df_musicas['track_uri'] = df_musicas['track_uri'].str.replace('spotify:track:', '')
df_musicas['artist_uri'] = df_musicas['artist_uri'].str.replace('spotify:artist:', '')
df_musicas['album_uri'] = df_musicas['album_uri'].str.replace('spotify:album:', '')

df_musicas.head(5)

,track_uri,artist_uri,album_uri
0,0UaMYEvWZi0ZqiDOoHU3YI,2wIVse2owClT7go1WT98tk,6vV5UrXcfyQD1wu4Qo2I9K
0,6I9VzXrHxO9rA9A5euc8Ak,26dSoYclwsYLMAKD3tpOr4,0z7pVBGOD7HCIB7S8eLkLI
0,0WqIKmW4BTrj3eJFmnCKMv,6vWDO969PvNqNYHIOW5v0m,25hVFAxTlDvXbx2X2QkUkE
0,1AWQoqb9bSvzTjaLralEkT,31TPClRtHm23RisEBtV3X7,6QPkyl04rXwTGlGlcYaRoW
0,1lzr43nnXAijIGYnCT8M8H,5EvFsr3kj42KNv97ZEnqij,6NmFmPX56pcLBOFMhIiKvF


In [15]:
tracks_ids = df_musicas['track_uri'].unique().tolist()
len(tracks_ids)

51

In [7]:
# scripts/api_spotify.py

def make_req(route: str, id: str) -> requests.models.Response:
    url = f'https://api.spotify.com/v1/{route}/{id}'

    headers = {
        'Authorization': f'Bearer {API_TOKEN}'
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f'Erro: {response.status_code}')
        print(f'Erro: {response.json()}')
        import sys
        sys.exit(1)

    else:
        return response


def get_track(id_track: str) -> pd.DataFrame:
    data = make_req('tracks', id_track).json()
    df = pd.DataFrame(data['tracks'])
    return df


def get_audio_features(id_track: str) -> dict:
    data = make_req('audio-features', id_track).json()
    df = pd.DataFrame(data['audio_features'])
    return df


def get_album(id_album: str) -> dict:

    return make_req('albums', id_album).json()


def get_artist_data(id_artist: str) -> dict:

    return make_req('artists', id_artist).json()




In [28]:
def get_music_features(tracks_ids: list) -> dict:

    tracks_ids_str = ','.join(tracks_ids)
    tracks_ids_str = '?ids=' + tracks_ids_str
    # print(tracks_ids_str)

    # 1. Get track data
    tracks_list_df = get_track(tracks_ids_str)

    # Collect album ids
    tracks_list_df['id_album'] = tracks_list_df['album'].apply(lambda x: x['id'])

    # Collect artist ids
    tracks_list_df['id_artist'] = tracks_list_df['artists'].apply(lambda x: [d['id'] for d in x])

    # Expand artist ids to separate rows
    tracks_list_df = tracks_list_df.explode('id_artist')

    
    # 2. Get audio features
    df_audio_features = pd.DataFrame()
    for i in range(0, len(tracks_ids), 50):
        tracks_ids_str = ','.join(tracks_ids[i:i+50] )
        tracks_ids_str = '?ids=' + tracks_ids_str
        df_audio_features_aux = get_audio_features(tracks_ids_str)
        df_audio_features = pd.concat([df_audio_features, df_audio_features_aux], axis=0)

    # Filtering columns
    df_audio_features = df_audio_features[['id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]
    
    # Merge audio features with track data
    tracks_list_df = tracks_list_df.merge(df_audio_features, on='id', how='left')


    # 3. Get album data

    # 4. Get artist data

    # 5. Merge all data

    # 6. Set relevante columns
    # relevant_features = [
    #     'name', 'explicit', 'duration_ms', 
    #     'popularity', 'danceability', 'energy',
    #     'key', 'loudness', 'mode', 'speechiness', 'acousticness',
    #     'instrumentalness', 'liveness', 'valence', 'tempo',
    #     'time_signature'
    # ]

    # track_data = {k: track_data[k] for k in relevant_features}

    # relevant_features = [
    #     'name', 'explicit', 'duration_ms', 'popularity'
    # ]

    # tracks_list_data_final = []

    # for track in tracks_list_data['tracks']:
    #     track_filtered = {k: track[k] for k in relevant_features}
    #     tracks_list_data_final.append(track_filtered)


    tracks_list_df = tracks_list_df[[
        'id', 'id_album', 'id_artist', 'name', 'explicit', 'duration_ms', 'popularity', # Track data
        'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence' # Audio features
        ]]

    return tracks_list_df


In [30]:
# a cada 50 musicas, pega os dados

df_musics = pd.DataFrame()
for i in range(0, len(tracks_ids), 50):
    df_aux = get_music_features(tracks_ids[i:i+50])
    df_musics = pd.concat([df_musics, df_aux], axis=0)

df_musics.iloc[0]

id                                      0UaMYEvWZi0ZqiDOoHU3YI
id_album                                6vV5UrXcfyQD1wu4Qo2I9K
id_artist                               2wIVse2owClT7go1WT98tk
name                Lose Control (feat. Ciara & Fat Man Scoop)
explicit                                                  True
duration_ms                                             226863
popularity                                                  69
acousticness                                            0.0311
danceability                                             0.904
energy                                                   0.813
instrumentalness                                       0.00697
key                                                          4
liveness                                                0.0471
loudness                                                -7.105
mode                                                         0
speechiness                                            

In [19]:
len(tracks_ids)

51

In [22]:
df_audio_features = pd.DataFrame()
for i in range(0, len(tracks_ids), 50):
    tracks_ids_str = ','.join(tracks_ids[i:i+50] )
    tracks_ids_str = '?ids=' + tracks_ids_str
    df_audio_features_aux = get_audio_features(tracks_ids_str)
    df_audio_features = pd.concat([df_audio_features, df_audio_features_aux], axis=0)

df_audio_features

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.904,0.813,4,-7.105,0,0.1210,0.031100,0.006970,0.0471,0.810,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4
1,0.774,0.838,5,-3.914,0,0.1140,0.024900,0.025000,0.2420,0.924,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800,4
2,0.664,0.758,2,-6.583,0,0.2100,0.002380,0.000000,0.0598,0.701,99.259,audio_features,0WqIKmW4BTrj3eJFmnCKMv,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933,4
3,0.892,0.714,4,-6.055,0,0.1410,0.201000,0.000234,0.0521,0.817,100.972,audio_features,1AWQoqb9bSvzTjaLralEkT,spotify:track:1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267,4
4,0.853,0.606,0,-4.596,1,0.0713,0.056100,0.000000,0.3130,0.654,94.759,audio_features,1lzr43nnXAijIGYnCT8M8H,spotify:track:1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600,4
5,0.881,0.788,2,-4.669,1,0.1680,0.021200,0.000000,0.0377,0.592,104.997,audio_features,0XUfyU2QviPAs6bxSpXYG4,spotify:track:0XUfyU2QviPAs6bxSpXYG4,https://api.spotify.com/v1/tracks/0XUfyU2QviPA...,https://api.spotify.com/v1/audio-analysis/0XUf...,250373,4
6,0.662,0.507,5,-8.238,1,0.1180,0.257000,0.000000,0.0465,0.676,86.412,audio_features,68vgtRHr7iZHpzGpon6Jlo,spotify:track:68vgtRHr7iZHpzGpon6Jlo,https://api.spotify.com/v1/tracks/68vgtRHr7iZH...,https://api.spotify.com/v1/audio-analysis/68vg...,223440,4
7,0.570,0.821,2,-4.380,1,0.2670,0.178000,0.000000,0.2890,0.408,210.857,audio_features,3BxWKCI06eQ5Od8TY2JBeA,spotify:track:3BxWKCI06eQ5Od8TY2JBeA,https://api.spotify.com/v1/tracks/3BxWKCI06eQ5...,https://api.spotify.com/v1/audio-analysis/3BxW...,225560,4
8,0.713,0.678,5,-3.525,0,0.1020,0.273000,0.000000,0.1490,0.734,138.009,audio_features,7H6ev70Weq6DdpZyyTmUXk,spotify:track:7H6ev70Weq6DdpZyyTmUXk,https://api.spotify.com/v1/tracks/7H6ev70Weq6D...,https://api.spotify.com/v1/audio-analysis/7H6e...,271333,4
9,0.727,0.974,4,-2.261,0,0.0664,0.103000,0.000532,0.1740,0.965,79.526,audio_features,2PpruBYCo4H7WOBJ7Q2EwM,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM,https://api.spotify.com/v1/tracks/2PpruBYCo4H7...,https://api.spotify.com/v1/audio-analysis/2Ppr...,235213,4


In [ ]:

get_audio_features(tracks_ids).head(1)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.904,0.813,4,-7.105,0,0.121,0.0311,0.00697,0.0471,0.81,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4


In [ ]:
relevant_features = [
    'name', 'explicit', 'duration_ms', 'popularity'
]

final_data = []

for track in tracks_list_data['tracks']:
    track_filtered = {k: track[k] for k in relevant_features}
    final_data.append(track_filtered)
    

final_data = pd.DataFrame(final_data)
final_data

,name,explicit,duration_ms,popularity
0,Lose Control (feat. Ciara & Fat Man Scoop),True,226863,69
1,Toxic,False,198800,84
2,Crazy In Love (feat. Jay-Z),False,235933,21
3,Rock Your Body,False,267266,79
4,It Wasn't Me,False,227600,0
5,Yeah!,False,250373,0
6,My Boo,False,223440,80
7,Buttons,False,225560,67
8,Say My Name,False,271333,79
9,Hey Ya!,False,235213,83


In [ ]:
tracks_list_data['tracks'][0]

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3ao3jf5d70Tf4fPh2bnXVl'},
    'href': 'https://api.spotify.com/v1/artists/3ao3jf5d70Tf4fPh2bnXVl',
    'id': '3ao3jf5d70Tf4fPh2bnXVl',
    'name': 'We The Kings',
    'type': 'artist',
    'uri': 'spotify:artist:3ao3jf5d70Tf4fPh2bnXVl'}],
  'available_markets': ['GB', 'IE'],
  'external_urls': {'spotify': 'https://open.spotify.com/album/2F1hfUOuMnOxtSfrktL8VX'},
  'href': 'https://api.spotify.com/v1/albums/2F1hfUOuMnOxtSfrktL8VX',
  'id': '2F1hfUOuMnOxtSfrktL8VX',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b27328b45bfdfd877c8040de8f36',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e0228b45bfdfd877c8040de8f36',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d0000485128b45bfdfd877c8040de8f36',
    'width': 64}],
  'name': 'We The Kings',
  'release_date': '2007',
  'relea

In [ ]:

def get_music_features(id_music: str) -> dict:
    
    # 1. Get track data
    track_data = get_track(id_music)

    # 2. Get audio features
    audio_features = get_audio_features(id_music)

    # 3. Merge data
    track_data.update(audio_features)

    # 4. Define relevant features
    relevant_features = [
        'name', 'explicit', 'duration_ms', 
        'popularity', 'danceability', 'energy',
        'key', 'loudness', 'mode', 'speechiness', 'acousticness',
        'instrumentalness', 'liveness', 'valence', 'tempo',
        'time_signature'
    ]

    track_data = {k: track_data[k] for k in relevant_features}

    return track_data


def get_album_features(id_album: str) -> dict:

    # 1. Get album data
    album_data = get_album(id_album)

    # 2. Define relevant features
    relevant_features = [
        'popularity', 'name', 'release_date', 'total_tracks', 'type'
    ]

    album_data = {k: album_data[k] for k in relevant_features}

    # 3. Add "album_" prefix to each key
    album_data = {f'album_{k}': v for k, v in album_data.items()}

    return album_data


def get_artist_features(id_artist: str) -> dict:
    # pass # get_artist_data

    # 1. Get artist data
    artist_data = get_artist_data(id_artist)

    # 2. Define relevant features
    relevant_features = [
        'genres', 'popularity', 'name', 'type', 'followers'
    ]

    artist_data = {k: artist_data[k] for k in relevant_features}

    # 3. At artist_followers key, keep only total
    artist_data['followers'] = artist_data['followers']['total']

    # 4. Add "artist_" prefix to each key
    artist_data = {f'artist_{k}': v for k, v in artist_data.items()}

    return artist_data

In [ ]:
df_musicas = df_musicas.sample(1)
df_musicas = df_musicas[['track_uri', 'artist_uri', 'album_uri']]

# Adiciona as features de música
df_musicas = df_musicas.merge(
    df_musicas['track_uri'].apply(get_music_features).apply(pd.Series),
    left_index=True,
    right_index=True
)

# # Adiciona as features de álbum
# df_musicas = df_musicas.merge(
#     df_musicas['album_uri'].apply(get_album_features).apply(pd.Series),
#     left_index=True,
#     right_index=True
# )

# # Adiciona as features de artista
# df_musicas = df_musicas.merge(
#     df_musicas['artist_uri'].apply(get_artist_features).apply(pd.Series),
#     left_index=True,
#     right_index=True
# )


In [ ]:
df_musicas

,track_uri,artist_uri,album_uri,name,explicit,duration_ms,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,2eJ8ij1T3cNUKiGdcUvKhy,1uNFoZAHBGtllmzznpCI3s,6gdLfnf2vdNlMTyhJHaDLs,Baby,False,213973,0,0.73,0.848,5,-5.262,0,0.101,0.0494,0,0.133,0.559,65.025,4


In [ ]:
# df_musicas.to_csv('df_musicas.csv', index=False)